# 1. 🤗 Set up a Postgres database to store the Huggingface Climate Policy Radar dataset.

Refer to README.md for instructions on setting up the Postgres database.

In [ ]:
import os
import regex as re
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import pgai
import torch
import glob
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
from datasets import load_dataset, Features, Value
from functions import generate_embeddings_for_text
from sqlalchemy import create_engine, text

tqdm.pandas()

## 1. Load the Huggingface dataset.

In [5]:
# Login using e.g. `huggingface-cli login` in the command line to access this dataset

ds = load_dataset("ClimatePolicyRadar/all-document-text-data")
ds = ds["train"]
flat_ds = ds.flatten()

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

## 2. Save 100000 chunks in the "climate_policy_radar" table.

The table contains all documents from the Climate Policy Radar dataset, with each row representing a chunk and all associated metadata stored in individual columns.

In [ ]:
def store_database_batched(flat_ds, batch_size=10000):
    """
    Store the flattened dataset into a PostgreSQL database using SQLAlchemy in chunks with progress bar.
    
    Args:
        flat_ds (list): Flattened dataset to be stored.
        chunk_size (int): Number of rows per chunk for batch insertion.
    """
    load_dotenv()
    from tqdm.notebook import tqdm as notebook_tqdm

    df = pd.DataFrame(flat_ds)
    engine = create_engine(os.getenv("DB_URL"))

    # Initialize table with first chunk
    df.iloc[:0].to_sql('climate_policy_radar', engine, if_exists='replace', index=False)

    # Iterate through chunks with progress bar
    for i in notebook_tqdm(range(0, len(df), batch_size), desc="Inserting chunks to DB"):
        chunk = df.iloc[i:i+batch_size]
        chunk.to_sql('climate_policy_radar', engine, if_exists='append', index=False)

In [ ]:
store_database_batched(flat_ds)